## Experiments: Sensitive Analysis Hyperparameter RSAST:


It is runned RSAST in a set of UCR datasets with a predefined number of runs ("runs"). Then, it is selected a range ("range_total") between 1, 10, 30 ,50 and 100 for the selected dataset.

In [195]:
import sys 
import os 
#add sast library path
file_path = os.path.dirname(os.getcwd())+"/sast"

#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.path.dirname(os.getcwd())+"\sast"


#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"/sast"


#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"\sast"


#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

#add cd_diagram library path
file_path = os.path.dirname(os.getcwd())+"\cd_diagram"

#file_path = r"C:\Users\Public\random_sast\cd_diagram"
sys.path.append(file_path)


file_path = os.path.dirname(os.getcwd())+"/cd_diagram"
#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"\cd_diagram"
#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"/cd_diagram"
#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

sys.path


['/home/nirojasvar/random_sast/ExperimentationRSAST',
 '/home/nirojasvar/anaconda3/envs/rsast_env/lib/python310.zip',
 '/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10',
 '/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/lib-dynload',
 '',
 '/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages',
 '/home/nirojasvar/random_sast/sast',
 '/home/nirojasvar/random_sast\\sast',
 '/home/nirojasvar/random_sast/ExperimentationRSAST/sast',
 '/home/nirojasvar/random_sast/ExperimentationRSAST\\sast',
 '/home/nirojasvar/random_sast\\cd_diagram',
 '/home/nirojasvar/random_sast/cd_diagram',
 '/home/nirojasvar/random_sast/ExperimentationRSAST\\cd_diagram',
 '/home/nirojasvar/random_sast/ExperimentationRSAST/cd_diagram',
 '/home/nirojasvar/random_sast/sast',
 '/home/nirojasvar/random_sast\\sast',
 '/home/nirojasvar/random_sast/ExperimentationRSAST/sast',
 '/home/nirojasvar/random_sast/ExperimentationRSAST\\sast',
 '/home/nirojasvar/random_sast\\cd_diagram',
 '/home/nir

In [196]:
from sast import *
import pandas as pd
import researchpy
import math
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import seaborn as sns

### Reading Datasets

In [197]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsByClassifier'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        # Append the dataframe to the list
        df['filename']=filename
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_other_methods = pd.concat(dfs, ignore_index=True)
df_other_methods=df_other_methods[['filename','folds:','0']]
df_other_methods['method']=df_other_methods['filename'].str.split('_').str[0]
df_other_methods
df_other_methods=df_other_methods.rename(columns={'folds:':'dataset','0':'score'})
df_other_methods=df_other_methods[['dataset','score','method']]


In [198]:
df_other_methods['method'].unique()

array(['SAST', 'ROCKET', 'HIVE-COTEv1', 'BOSS', 'RISE', 'TSF',
       'InceptionTime', 'TS-CHIEF', 'S-BOSS', 'STC', 'ResNet',
       'ProximityForest', 'WEASEL', 'Catch22', 'cBOSS'], dtype=object)

In [199]:
# Set directory where the csv files are located
#directory = os.getcwd()+'/ResultsRsast'
#directory = os.getcwd()+'/results_accuracy_per_ds'

directories=[]
directories.append(os.getcwd()+'/ResultsRsast/Server17_Comparison_RSAST')
#directories.append(os.getcwd()+'/ResultsRsast/Server16_Hyperparameter_Tunning')
#directories.append(os.getcwd()+'/ResultsRsast/Server17_Hyperparameter_Tunning')
#directories.append(os.getcwd()+'/results_accuracy_per_ds')
#directories.append(os.getcwd()+'/results_accuracy_per_ds_10000')


# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for directory in directories:
    for filename in os.listdir(directory):
        # Check if the file starts with "df_overall_tunning" and ends with ".csv"
        if filename.startswith("df_all_overall_tunning") and filename.endswith(".csv"):
            # Read the csv file into a dataframe
            filepath = os.path.join(directory, filename)
            df = pd.read_csv(filepath)
            df['rpoint']=df['rpoint'].astype(str)
            df['nb_per_class']=df['nb_per_class'].astype(str)
            df['rpoint']=df['rpoint'].replace("(lenthg ts)//2","half_len")
            df['nb_per_class']=df['nb_per_class'].replace("(max instances per class)//2","half_instance")
            df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
            df['classifier_name']=df['classifier_name'].str.replace("\(max instances per class\)//2","half_instance")
            # Append the dataframe to the list
            dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_result = pd.concat(dfs, ignore_index=True)
# df_result.head(3)


/tmp/ipykernel_2684/218631293.py:28: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
/tmp/ipykernel_2684/218631293.py:29: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(max instances per class\)//2","half_instance")
/tmp/ipykernel_2684/218631293.py:28: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
/tmp/ipykernel_2684/218631293.py:29: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(max instances per class\)//2","half_instance")
/tmp/ipykernel_2684/218631293.py:28: FutureWarning: The default value of regex wil

In [200]:
"""
len_methods=['ACF&PACF: n_random_points=30 nb_inst_per_class=1',
'ACF&PACF: n_random_points=10 nb_inst_per_class=10',
'None: n_random_points=30 nb_inst_per_class=1',
'None: n_random_points=10 nb_inst_per_class=10'
]
df_result=df_result[df_result.classifier_name.isin(len_methods)]
"""

"\nlen_methods=['ACF&PACF: n_random_points=30 nb_inst_per_class=1',\n'ACF&PACF: n_random_points=10 nb_inst_per_class=10',\n'None: n_random_points=30 nb_inst_per_class=1',\n'None: n_random_points=10 nb_inst_per_class=10'\n]\ndf_result=df_result[df_result.classifier_name.isin(len_methods)]\n"

In [201]:
#create dataset to generate cd diagram
#df_perf=pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
#df_perf=df_perf.reset_index()
#df_perf.classifier_name.unique()

In [202]:
"""
#filter dataset for cd-diagrams with the ds that has already tested all combinations of params
param_per_ds=pd.pivot_table(df_perf, index=['dataset_name'],values=['classifier_name'] ,aggfunc='count')
param_per_ds=param_per_ds.reset_index()
complete_ds=param_per_ds[param_per_ds.classifier_name==max(param_per_ds.classifier_name)]["dataset_name"].unique()

complete_ds=complete_ds.tolist()

#complete_ds.remove('DistalPhalanxOutlineCorrect')

#df_perf=df_perf[df_perf.dataset_name.isin(complete_ds)]
#df_result=df_result[df_result.dataset_name.isin(complete_ds)]
"""


'\n#filter dataset for cd-diagrams with the ds that has already tested all combinations of params\nparam_per_ds=pd.pivot_table(df_perf, index=[\'dataset_name\'],values=[\'classifier_name\'] ,aggfunc=\'count\')\nparam_per_ds=param_per_ds.reset_index()\ncomplete_ds=param_per_ds[param_per_ds.classifier_name==max(param_per_ds.classifier_name)]["dataset_name"].unique()\n\ncomplete_ds=complete_ds.tolist()\n\n#complete_ds.remove(\'DistalPhalanxOutlineCorrect\')\n\n#df_perf=df_perf[df_perf.dataset_name.isin(complete_ds)]\n#df_result=df_result[df_result.dataset_name.isin(complete_ds)]\n'

In [203]:
#get ds tested overall
df_result['dataset_name'].unique()

array(['MiddlePhalanxOutlineAgeGroup', 'Wafer',
       'DistalPhalanxOutlineAgeGroup', 'ECG200', 'ItalyPowerDemand',
       'TwoLeadECG', 'ProximalPhalanxOutlineCorrect', 'CBF',
       'TwoPatterns', 'UMD', 'Strawberry', 'FaceAll',
       'GunPointMaleVersusFemale', 'GunPointOldVersusYoung', 'PowerCons',
       'SonyAIBORobotSurface2', 'ProximalPhalanxOutlineAgeGroup', 'Wine',
       'MiddlePhalanxOutlineCorrect', 'ProximalPhalanxTW', 'SwedishLeaf',
       'SonyAIBORobotSurface1', 'PhalangesOutlinesCorrect', 'ECG5000',
       'GunPoint', 'SmoothSubspace', 'ChlorineConcentration',
       'ElectricDevices', 'GunPointAgeSpan', 'ECGFiveDays', 'Chinatown',
       'Adiac', 'DistalPhalanxOutlineCorrect', 'Plane', 'MiddlePhalanxTW',
       'MoteStrain', 'Crop', 'SyntheticControl', 'InsectWingbeatSound',
       'DistalPhalanxTW', 'Fungi', 'FacesUCR', 'ArrowHead',
       'MedicalImages', 'BME'], dtype=object)

In [204]:
#get columns of generated datasets
df_result.columns.unique()

Index(['Unnamed: 0', 'accuracy', 'time', 'cweights_time', 'fsubsequence_time',
       'tdataset_time', 'tclassifier_time', 'dataset_name', 'classifier_name',
       'rpoint', 'nb_per_class', 'method', 'len_method'],
      dtype='object')

### Hyperparameter Tunning RSAST 

#### Accuracy: Subsequence Lenght Method 

In [205]:
filter_param=df_result

#filter_param=filter_param[filter_param.len_method=='ACF&PACF']
filter_param.dataset_name.unique()

array(['MiddlePhalanxOutlineAgeGroup', 'Wafer',
       'DistalPhalanxOutlineAgeGroup', 'ECG200', 'ItalyPowerDemand',
       'TwoLeadECG', 'ProximalPhalanxOutlineCorrect', 'CBF',
       'TwoPatterns', 'UMD', 'Strawberry', 'FaceAll',
       'GunPointMaleVersusFemale', 'GunPointOldVersusYoung', 'PowerCons',
       'SonyAIBORobotSurface2', 'ProximalPhalanxOutlineAgeGroup', 'Wine',
       'MiddlePhalanxOutlineCorrect', 'ProximalPhalanxTW', 'SwedishLeaf',
       'SonyAIBORobotSurface1', 'PhalangesOutlinesCorrect', 'ECG5000',
       'GunPoint', 'SmoothSubspace', 'ChlorineConcentration',
       'ElectricDevices', 'GunPointAgeSpan', 'ECGFiveDays', 'Chinatown',
       'Adiac', 'DistalPhalanxOutlineCorrect', 'Plane', 'MiddlePhalanxTW',
       'MoteStrain', 'Crop', 'SyntheticControl', 'InsectWingbeatSound',
       'DistalPhalanxTW', 'Fungi', 'FacesUCR', 'ArrowHead',
       'MedicalImages', 'BME'], dtype=object)

In [206]:
filter_param.rpoint.unique()

array(['10', '30'], dtype=object)

In [207]:
#filter_param=filter_param[filter_param.rpoint=='half_len']

In [208]:
filter_param.nb_per_class.unique()

array(['1', '10'], dtype=object)

In [209]:
#filter_param=filter_param[filter_param.nb_per_class=='30']

In [210]:
# create a pivot table with the mean of score by dataset
len_method_vs_ds=pd.pivot_table(filter_param, values='accuracy', index=['len_method'],columns=['dataset_name'], aggfunc='mean')
len_method_vs_ds=np.transpose(len_method_vs_ds)#[['ACF','PACF']]
len_method_vs_ds=len_method_vs_ds.reset_index()
df_rocket=df_other_methods[df_other_methods["method"]=="SAST"]
merged_df = len_method_vs_ds.merge(df_rocket,left_on='dataset_name', right_on='dataset',  how='left')
merged_df=merged_df.drop('dataset',axis=1)
merged_df


,dataset_name,ACF&PACF,Max PACF,None,score,method
0,Adiac,0.596036,0.537468,0.639386,0.68,SAST
1,ArrowHead,0.770286,0.705714,0.690000,0.77,SAST
2,BME,0.829333,0.779333,0.813667,0.87,SAST
3,CBF,0.984778,0.958889,0.902944,0.98,SAST
4,Chinatown,0.969242,0.963703,0.900437,0.96,SAST
5,ChlorineConcentration,0.735807,0.604427,0.615508,0.75,SAST
6,Crop,0.687699,0.703176,0.713351,0.73,SAST
7,DistalPhalanxOutlineAgeGroup,0.736691,0.743525,0.735252,0.76,SAST
8,DistalPhalanxOutlineCorrect,0.730978,0.722464,0.740761,0.74,SAST
9,DistalPhalanxTW,0.631295,0.648561,0.656475,0.67,SAST


In [215]:
merged_df.describe()

,ACF&PACF,Max PACF,None,score
count,45.000000,45.000000,45.000000,45.000000
mean,0.827410,0.796931,0.799094,0.847111
std,0.140727,0.137924,0.125720,0.131954
min,0.508611,0.461288,0.493763,0.530000
25%,0.770286,0.713810,0.728120,0.760000
50%,0.859000,0.811621,0.814722,0.870000
75%,0.949525,0.934022,0.902944,0.960000
max,1.000000,0.990712,0.989048,1.000000


In [213]:

# Summary statistics for a Series (single variable)

summ_ds=filter_param
summ_ds=researchpy.summary_cont(summ_ds.groupby(['dataset_name'])['accuracy'], conf = 0.95)
summ_ds

,N,Mean,SD,SE,95% Conf.,Interval
dataset_name,,,,,,
Adiac,60,0.5910,0.0768,0.0099,0.5711,0.6108
ArrowHead,60,0.7220,0.0562,0.0073,0.7075,0.7365
BME,60,0.8074,0.0544,0.0070,0.7934,0.8215
CBF,60,0.9489,0.0601,0.0078,0.9333,0.9644
Chinatown,60,0.9445,0.0852,0.0110,0.9224,0.9665
ChlorineConcentration,60,0.6519,0.0817,0.0105,0.6308,0.6730
Crop,60,0.7014,0.0306,0.0039,0.6935,0.7093
DistalPhalanxOutlineAgeGroup,60,0.7385,0.0203,0.0026,0.7332,0.7437
DistalPhalanxOutlineCorrect,60,0.7314,0.0248,0.0032,0.7250,0.7378


In [171]:
stats = filter_param.groupby(['len_method'])['accuracy'].agg(['mean', 'count', 'std'])

ci95_hi = []
ci95_lo = []

for i in stats.index:
    m, c, s = stats.loc[i]
    ci95_hi.append(m + 1.96*s/math.sqrt(c))
    ci95_lo.append(m - 1.96*s/math.sqrt(c))

stats['ci95_hi'] = ci95_hi
stats['ci95_lo'] = ci95_lo
print(stats.head(10))

                mean  count       std   ci95_hi   ci95_lo
len_method                                               
ACF&PACF    0.827410    900  0.145665  0.836927  0.817894
Max PACF    0.796931    900  0.147371  0.806560  0.787303
None        0.799094    900  0.139721  0.808222  0.789965


#### Generate Boxplot Tunning Hyperparameter

In [172]:
df_result.nb_per_class.unique()

array(['1', '10'], dtype=object)

In [173]:
df_result.len_method.unique()

array(['ACF&PACF', 'Max PACF', 'None'], dtype=object)

In [174]:
#df_perf=pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name','rpoint'], aggfunc='mean')
#df_perf=df_perf.reset_index()
df_perf=df_result[["classifier_name","dataset_name","accuracy","len_method","rpoint","nb_per_class","time","cweights_time","fsubsequence_time", "tdataset_time"]]
df_perf.len_method.unique()
len_m="ACF&PACF"
ints="1"
ds="SyntheticControl"


df_to_cd=df_perf[df_perf.classifier_name.str.contains(r'^'+len_m+':')]
df_to_cd=df_to_cd[df_to_cd.dataset_name==ds]


max_bx=max(df_to_cd.accuracy)
min_bx=min(df_to_cd.accuracy)

df_to_cd=df_to_cd[df_to_cd.classifier_name.str.contains(r':.*nb_inst_per_class='+str(ints)+'$')] 
#df_to_cd

In [175]:
#generate hyperparameter tuning boxplots

for k, ints in enumerate(df_perf.nb_per_class.unique()):

    for len_m in df_perf.len_method.unique():
        
        
        df_to_cd=df_perf[df_perf.classifier_name.str.contains(r'^'+len_m+':.*nb_inst_per_class='+str(ints)+'$')]
        #df_to_cd=pd.pivot_table(df_to_cd, index=['dataset_name','classifier_name'], columns=['rpoint'],values='accuracy')
        #df_to_cd = df_to_cd.reindex(columns=["1","10","30","50","100","half_len"])
        
        #print(df_to_cd.head(5))

        # Plot
        fig, ax = plt.subplots()
        
        
        #order=list(df_to_cd)
        sns.boxplot(data=df_to_cd, x='rpoint', y='accuracy', palette="Blues")
        #plt.boxplot(df_to_cd, labels=list(df_to_cd), showfliers=False)
        
        max_bx=max(df_to_cd.accuracy)
        min_bx=min(df_to_cd.accuracy)
        ax.set_ylim(min_bx,max_bx)
        # Axis details
        ax.set(xlabel='number of random points', ylabel='accuracy', title='Accuracy Boxplot for Len method:'+len_m+' and N° of Instances:'+ints)
        #plt.xticks([1, 2, 3, 4, 5, 6],list(df_to_cd) )
        #print(df_to_cd.describe())

        # save plot
        plt.savefig('images_boxplot_acc/boxplot_acc'+len_m+ints+'.png')




In [176]:
#generate hyperparameter tuning boxplots
for ds in df_perf.dataset_name.unique():
    for k, ints in enumerate(df_perf.nb_per_class.unique()):
        for len_m in df_perf.len_method.unique():
        

            df_to_cd=df_perf[df_perf.classifier_name.str.contains(r'^'+len_m+':')]
            df_to_cd=df_to_cd[df_to_cd.dataset_name==ds]

            df_to_cd["time"]=df_to_cd['cweights_time']+df_to_cd['fsubsequence_time']+df_to_cd['tdataset_time']
            df_to_cd["time"]=df_to_cd["time"]/60
            max_bx=max(df_to_cd.accuracy)
            min_bx=min(df_to_cd.accuracy)
            
            df_to_cd=df_to_cd[df_to_cd.classifier_name.str.contains(r':.*nb_inst_per_class='+str(ints)+'$')]    

            #df_to_cd=pd.pivot_table(df_to_cd, index=['dataset_name','classifier_name'], columns=['rpoint'],values='accuracy')
            #df_to_cd = df_to_cd.reindex(columns=["1","10","30","50","100","half_len"])
            
            #print(df_to_cd.head(5))

            # Plot
            fig, ax = plt.subplots()
            
            
            #order=list(df_to_cd)
            sns.boxplot(data=df_to_cd, x='rpoint', y='accuracy', palette="Blues")
            #plt.boxplot(df_to_cd, labels=list(df_to_cd), showfliers=False)
            
            ax.set_ylim(min_bx,max_bx)
            # Axis details
            ax.set(xlabel='number of random points', ylabel='accuracy', title='Accuracy Boxplot for Len method:'+len_m+' and N° of Instances:'+ints)
            #plt.xticks([1, 2, 3, 4, 5, 6],list(df_to_cd) )
            #print(df_to_cd.describe())

            # save plot
            plt.savefig('images_boxplot_acc_per_ds/boxplot_acc'+len_m+ints+'_'+ds+'.png')

/tmp/ipykernel_2684/2053361988.py:30: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax.set_ylim(min_bx,max_bx)
/tmp/ipykernel_2684/2053361988.py:30: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax.set_ylim(min_bx,max_bx)


In [177]:
#generate hyperparameter tuning boxplots

for k, ints in enumerate(df_perf.nb_per_class.unique()):
    for len_m in df_perf.len_method.unique():
        
        
        df_to_cd=df_perf[df_perf.classifier_name.str.contains(r'^'+len_m+':.*nb_inst_per_class='+str(ints)+'$')]
        df_to_cd["time"]=df_to_cd['cweights_time']+df_to_cd['fsubsequence_time']+df_to_cd['tdataset_time']
        df_to_cd["time"]=df_to_cd["time"]/60
        #df_to_cd=pd.pivot_table(df_to_cd, index=['dataset_name','classifier_name'], columns=['rpoint'],values='accuracy')
        #df_to_cd = df_to_cd.reindex(columns=["1","10","30","50","100","half_len"])
        
        #print(df_to_cd.head(5))

        # Plot
        fig, ax = plt.subplots()
        
        
        #order=list(df_to_cd)
        sns.boxplot(data=df_to_cd, x='rpoint', y='time', palette="Blues")
        #plt.boxplot(df_to_cd, labels=list(df_to_cd), showfliers=False)
        
        max_bx=max(df_to_cd.time)
        min_bx=min(df_to_cd.time)
        ax.set_ylim(min_bx,max_bx)
        # Axis details
        ax.set(xlabel='number of random points', ylabel='time', title='Time Boxplot for Len method:'+len_m+' and N° of Instances:'+ints)
        #plt.xticks([1, 2, 3, 4, 5, 6],list(df_to_cd) )
        #print(df_to_cd.describe())

        # save plot
        plt.savefig('images_boxplot_time/boxplot_time'+len_m+ints+'.png')




/tmp/ipykernel_2684/981854558.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_cd["time"]=df_to_cd['cweights_time']+df_to_cd['fsubsequence_time']+df_to_cd['tdataset_time']
/tmp/ipykernel_2684/981854558.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_cd["time"]=df_to_cd["time"]/60
/tmp/ipykernel_2684/981854558.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

In [178]:
#generate hyperparameter tuning boxplots
for ds in df_perf.dataset_name.unique():
    for k, ints in enumerate(df_perf.nb_per_class.unique()):
        for len_m in df_perf.len_method.unique():
        


            df_to_cd=df_perf[df_perf.classifier_name.str.contains(r'^'+len_m+':')]
            df_to_cd=df_to_cd[df_to_cd.dataset_name==ds]

            df_to_cd["time"]=df_to_cd['cweights_time']+df_to_cd['fsubsequence_time']+df_to_cd['tdataset_time']
            df_to_cd["time"]=df_to_cd["time"]/60
            max_bx=max(df_to_cd.time)
            min_bx=min(df_to_cd.time)

            df_to_cd=df_to_cd[df_to_cd.classifier_name.str.contains(r':.*nb_inst_per_class='+str(ints)+'$')]


            #df_to_cd=pd.pivot_table(df_to_cd, index=['dataset_name','classifier_name'], columns=['rpoint'],values='accuracy')
            #df_to_cd = df_to_cd.reindex(columns=["1","10","30","50","100","half_len"])
            
            #print(df_to_cd.head(5))

            # Plot
            fig, ax = plt.subplots()
            
            
            #order=list(df_to_cd)
            sns.boxplot(data=df_to_cd, x='rpoint', y='time', palette="Blues")
            #plt.boxplot(df_to_cd, labels=list(df_to_cd), showfliers=False)
            

            ax.set_ylim(min_bx,max_bx)
            # Axis details
            ax.set(xlabel='number of random points', ylabel='time', title='Time Boxplot for Len method:'+len_m+' and N° of Instances:'+ints)
            #plt.xticks([1, 2, 3, 4, 5, 6],list(df_to_cd) )
            #print(df_to_cd.describe())

            # save plot
            plt.savefig('images_boxplot_time_per_ds/boxplot_time'+len_m+ints+'_'+ds+'.png')


#### Generate CD Diagram Tunning Hyperparameter

In [179]:
df_perf=pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
df_perf=df_perf.reset_index()
df_perf.classifier_name.unique()

array(['ACF&PACF: n_random_points=10 nb_inst_per_class=1',
       'ACF&PACF: n_random_points=10 nb_inst_per_class=10',
       'ACF&PACF: n_random_points=30 nb_inst_per_class=1',
       'ACF&PACF: n_random_points=30 nb_inst_per_class=10',
       'Max PACF: n_random_points=10 nb_inst_per_class=1',
       'Max PACF: n_random_points=10 nb_inst_per_class=10',
       'Max PACF: n_random_points=30 nb_inst_per_class=1',
       'Max PACF: n_random_points=30 nb_inst_per_class=10',
       'None: n_random_points=10 nb_inst_per_class=1',
       'None: n_random_points=10 nb_inst_per_class=10',
       'None: n_random_points=30 nb_inst_per_class=1',
       'None: n_random_points=30 nb_inst_per_class=10'], dtype=object)

In [180]:
len_m="None"
ints="10"
df_perf[df_perf.classifier_name.str.contains(r'^'+len_m+':.*$')].classifier_name.unique()

array(['None: n_random_points=10 nb_inst_per_class=1',
       'None: n_random_points=10 nb_inst_per_class=10',
       'None: n_random_points=30 nb_inst_per_class=1',
       'None: n_random_points=30 nb_inst_per_class=10'], dtype=object)

In [181]:


from cd_function import *
#generate hyperparameter tuning cd diagrams

best_comb_by_method=[]
best=""
for len_m in df_result.len_method.unique():
   df_to_cd=df_perf[df_perf.classifier_name.str.contains(r'^'+len_m+':.*$')]
   draw_cd_diagram(df_to_cd, labels=True, title=len_m +" comparison", name_file='images_cd_diagram/cd-diagram_'+len_m+'.png')
   _, average_ranks, _ = wilcoxon_holm(df_perf=df_to_cd)
   min_rank= min(average_ranks)
   average_ranks=pd.DataFrame(average_ranks)
   best=average_ranks[average_ranks[0]==min_rank][0].index
   best_comb_by_method.append(best[0])



/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not 

In [182]:
type(best_comb_by_method)

list

In [183]:

df_to_cd=df_perf[df_perf.classifier_name.isin(best_comb_by_method)]
draw_cd_diagram(df_to_cd, labels=True, title="Best combination comparison", name_file='images_cd_diagram/cd-diagram_best_com.png')


/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


In [184]:
draw_cd_diagram(df_perf, labels=True, name_file='images_cd_diagram/cd-diagram_all.png')


/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


#### Focus on most accurate lenght method

##### Overall Accuracy

In [185]:
# highest accurate hyperparameters
best_comb_by_method

['ACF&PACF: n_random_points=30 nb_inst_per_class=10',
 'Max PACF: n_random_points=30 nb_inst_per_class=10',
 'None: n_random_points=30 nb_inst_per_class=10']

In [186]:
# filter by method with ACF&PACF
# create a pivot table with the mean of score by hyperparameter
df_result_acc=df_result[df_result["len_method"]=="PACF"]
#df_result_acc=df_result_acc[df_result_acc["dataset_name"]=="Fungi"]

pivot = pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')
pivot = pivot.reindex(columns=["half_instance","1","10","30","50","100","1000","10000"], index=["half_len","1","10","30","50","100","1000","10000"])
pivot

nb_per_class,half_instance,1,10,30,50,100,1000,10000
rpoint,,,,,,,,
half_len,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [187]:
# create a pivot table with the variatioon of score by hyperparameter
pivot = pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='var')
pivot = pivot.reindex(columns=["half_instance","1","10","30","50","100","1000","10000"], index=["half_len","1","10","30","50","100","1000","10000"])
pivot

nb_per_class,half_instance,1,10,30,50,100,1000,10000
rpoint,,,,,,,,
half_len,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Overall time complexity

In [188]:
# create a pivot table with the mean of time spent in minutes
pivot=pd.pivot_table(df_result_acc, values=['time','cweights_time','fsubsequence_time','tdataset_time','tclassifier_time'], index=['dataset_name'], aggfunc='mean')/60
pivot = pivot.reindex(columns=['cweights_time','fsubsequence_time','tdataset_time','tclassifier_time','time'])
pivot

,cweights_time,fsubsequence_time,tdataset_time,tclassifier_time,time
dataset_name,,,,,


In [189]:
# create a pivot table with the mean of time spent in minutes by random points and instances per class
pivot = pd.pivot_table(df_result_acc, values='time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')/60
pivot = pivot.reindex(columns=["half_instance","1","10","30","50","100","1000","10000"], index=["half_len","1","10","30","50","100","1000","10000"])
pivot

nb_per_class,half_instance,1,10,30,50,100,1000,10000
rpoint,,,,,,,,
half_len,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Calculate weights time complexity

In [190]:
pd.pivot_table(df_result_acc, values='cweights_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class
rpoint


##### Finding subsequences time complexity

In [191]:
pd.pivot_table(df_result_acc, values='fsubsequence_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class
rpoint


##### Transform Dataset time complexity

In [192]:
pd.pivot_table(df_result_acc, values='tdataset_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class
rpoint


##### Classifier time complexity

In [193]:
pd.pivot_table(df_result_acc, values='tclassifier_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class
rpoint
